In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0022852420806884766
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 11.08143745112115
200000 22.095414360555075
300000 25.643206563198845
400000 25.62553432891649
500000 26.796147916465756
600000 26.579230769179926
700000 25.255874064960786
800000 25.12249776811348
900000 24.16607729813657
1000000 10.907250871663154
1100000 24.891734716122816
1200000 25.204404142377054
1300000 24.790704199845134
1400000 25.761885856569187
1500000 25.74620897012026
1600000 25.10485330238357
1700000 25.256826587568902
1800000 25.462962919021393
1900000 10.283381541767389
2000000 25.602376940296143
2100000 30.31822120541653
2200000 28.907929960413743
2300000 29.174101218947246
2400000 29.860090276677738
2500000 27.750351833006036
2600000 27.9722115244889
2700000 27.540500376973526
2800000 17.716677679894982
2900000 21.958286320480447
300000

29400000 23.219483143935054
29500000 23.130496899966495
29600000 11.870736285037987
29700000 23.377844460227177
29800000 23.533523507071227
29900000 23.499393317294164
30000000 21.294089630128024
30100000 12.723813016097008
30200000 23.641791361714933
30300000 22.61926958928538
30400000 22.674161195709924
30500000 21.935585977996062
30600000 20.086023594166672
30700000 15.804056474969446
30800000 14.56209759000144
30900000 20.621014776781546
31000000 21.63114356611328
31100000 22.783974491122827
31200000 22.80355822062625
31300000 23.630022711482784
31400000 14.892113121727196
31500000 20.59689880290205
31600000 23.15633875957212
31700000 23.51844118049368
31800000 22.90329465327677
31900000 16.922555725025894
32000000 17.435602406929075
32100000 23.370492851771093
32200000 22.020138798970237
32300000 22.6997575148796
32400000 20.609911775639976
32500000 20.071765966308206
32600000 8.456796805240039
32700000 20.945127734053937
32800000 20.1498152586044
32900000 21.86577044254134
330000

59200000 19.38172465936332
59300000 20.20370856173566
59400000 20.48265844261411
59500000 20.5167167623823
59600000 21.82330920153191
59700000 21.122696564944768
59800000 20.76253712325107
59900000 20.22508780488254
60000000 20.575038309256257
60100000 9.038536649097969
60200000 18.599623824760844
60300000 18.688981938093907
60400000 19.044053562846035
60500000 9.577720595835146
60600000 16.962140615328437
60700000 18.909134173381506
60800000 18.527516585082466
60900000 16.515088204899275
61000000 13.445759619129124
61100000 19.703311937854686
61200000 20.583400785712282
61300000 20.485684873485805
61400000 20.736809803065473
61500000 21.148353473063967
61600000 20.649727670496862
61700000 20.495904893818462
61800000 19.248676021946835
61900000 16.711289620812522
62000000 12.075064573592813
62100000 18.172574367788194
62200000 19.183018486191184
62300000 18.903490877562046
62400000 7.428933194439332
62500000 18.546804247460386
62600000 18.24570503019923
62700000 18.261367402140774
6280

88900000 17.069913516943146
89000000 16.504875513470715
89100000 16.991331342520876
89200000 18.08350066663292
89300000 18.43842379215836
89400000 18.88735495836471
89500000 18.553931736494004
89600000 17.37160825795945
89700000 16.701494874206997
89800000 16.80574468608145
89900000 17.423863150254324
90000000 16.52211567672706
90100000 8.191895322891474
90200000 16.423014552075028
90300000 6.285800781540149
90400000 17.155261631641928
90500000 8.66194661873913
90600000 17.004897595085065
90700000 17.01848115319412
90800000 16.91301479078337
90900000 16.775913132446842
91000000 17.051805193763673
91100000 18.651519085413383
91200000 18.501904532645746
91300000 19.118861388489535
91400000 18.001954244619043
91500000 16.566812423286347
91600000 16.559358695921272
91700000 16.86548446151193
91800000 16.931081227251866
91900000 16.01382672266629
92000000 8.373024486035467
92100000 13.496838527112821
92200000 10.243517918516696
92300000 11.324643346443139
92400000 13.471971587770796
9250000

118000000 1.7311326402669844
118100000 1.722269065315899
118200000 1.7051715631560518
118300000 1.695849327535848
118400000 1.7053380715513162
118500000 1.7243952577668826
118600000 1.7119684018680354
118700000 1.7329020363418923
118800000 1.7234417549175456
118900000 1.7056825876935422
119000000 1.7052473730547197
119100000 2.5336962018992306
119200000 22.752695275066085
119300000 21.323199360331756
119400000 21.163719999963647
119500000 20.482916849412728
119600000 20.620905213269808
119700000 20.123351670211555
119800000 19.746201672981293
119900000 20.262688350208997
120000000 10.571389513353894
120100000 2.6545119113235076
120200000 18.937722962263628
120300000 19.573037616510607
120400000 20.2461099302408
120500000 19.98279241912215
120600000 21.22727283791316
120700000 20.52944225920433
120800000 21.383067276751152
120900000 21.508968660221807
121000000 20.872243071416023
121100000 17.40973071284845
121200000 17.261903167617508
121300000 16.19181329015073
121400000 16.6803007902

...

233100000 2.1831128679469294
233200000 2.2026139584935605
233300000 2.0787526951917905
233400000 2.071460972953174
233500000 0.9931162621959945
233600000 0.5539944293891788
233700000 2.5784401391054526
233800000 2.225873776954452
233900000 1.911999514160852
234000000 2.645755616915829
234100000 2.1015188608906565
234200000 2.1101715462409287
234300000 2.1210822276655072
234400000 1.883093392233491
234500000 2.952953889196535
234600000 2.3041494573408032
234700000 1.8828804964219232
234800000 2.085341652416865
234900000 2.5344016948805588
235000000 2.0710476873494983
235100000 2.0372838640441295
235200000 1.9893624324211345
235300000 2.6207028273824653
235400000 0.5629093848096729
235500000 0.5168028175820377
235600000 2.1114435311185815
235700000 3.123885517201781
235800000 2.0478670970831976
235900000 3.1704991260293425
236000000 2.6990297974145476
236100000 1.9108818330704402
236200000 1.9059039096483785
236300000 2.204978257845183
236400000 2.34140864436389
236500000 2.2559783205025

261700000 3.083535988972972
261800000 3.331773577244805
261900000 3.159949110551473
262000000 3.110278166073024
262100000 3.1790066589935724
262200000 3.1194623264691628
262300000 3.17130863857306
262400000 3.867993056562318
262500000 3.80537230467637
262600000 3.424142732105245
262700000 3.240682699558447
262800000 3.2166079135773114
262900000 3.299156534588486
263000000 3.236152745946328
263100000 3.095069339059439
263200000 3.474574017354766
263300000 0.7409716857134236
263400000 0.748280562454684
263500000 2.7706635244365136
263600000 3.2361896232391563
263700000 3.2789254845689104
263800000 3.444550529929698
263900000 3.2075570185060998
264000000 2.9397944248252643
264100000 3.3245066048751473
264200000 3.28981686515175
264300000 4.465743297452294
264400000 4.049167018286924
264500000 3.598853580662873
264600000 3.3662040678270033
264700000 3.734521931769848
264800000 3.7431236121889158
264900000 3.5334769720554324
265000000 3.176936886116535
265100000 3.1810732662953174
265200000

290500000 2.132219238769243
290600000 2.129528009186768
290700000 1.9185513764016164
290800000 2.217192712945282
290900000 2.079797527720693
291000000 2.007875912293656
291100000 2.1294084414525924
291200000 0.7931948472690582
291300000 0.5022782626173165
291400000 1.858871236389051
291500000 1.9140340603878991
291600000 2.0138358596497543
291700000 2.1033501647510824
291800000 2.1308957948388954
291900000 1.9275404325581889
292000000 2.139749738807003
292100000 2.1808076454938816
292200000 2.454053463469923
292300000 2.1282017154183688
292400000 2.15901254457887
292500000 1.858436947309421
292600000 2.069542987523377
292700000 2.0809040979869895
292800000 2.158905721341968
292900000 1.971394053721855
293000000 1.700506946287188
293100000 0.48544048412197494
293200000 0.4838423118320902
293300000 2.332189015935378
293400000 1.9082452535115482
293500000 2.0565370409582555
293600000 2.017650400393973
293700000 2.1226976135299203
293800000 1.912585901239107
293900000 2.1905306515135665
29

318300000 1.138854087151438
318400000 1.2181183397488793
318500000 1.0896221905056436
318600000 1.2372542483452593
318700000 1.1729444973929524
318800000 1.166332590519405
318900000 1.1136335652337537
319000000 1.1523447673054834
319100000 1.088344536351588
319200000 1.2370226250100333
319300000 1.1417752473062674
319400000 1.1023886125976377
319500000 1.1143667246135374
319600000 1.094627649874717
319700000 1.116915531414492
319800000 1.2392724075878958
319900000 1.0941449755732517
320000000 1.1181997653238973
320100000 1.0708759970534412
320200000 1.068770017832309
320300000 1.1270647523857653
320400000 1.1511726135086318
320500000 1.139001389406681
320600000 1.0969742107129428
320700000 1.061370066875418
320800000 1.1393876359722914
320900000 1.0267322766895493
321000000 1.0970284104174675
321100000 1.1482287529539963
321200000 1.0584211873247191
321300000 1.065063362625397
321400000 0.9879200114614667
321500000 0.9702089023991259
321600000 1.0661576810031632
321700000 1.07427670250

345300000 0.23618654247229962
345400000 0.15880378005453943
345500000 0.2151874968822797
345600000 0.1737787261063059
345700000 0.1920561357549131
345800000 0.1718961784960197
345900000 0.16507849651041956
346000000 0.17329235373603305
346100000 0.14450930626156597
346200000 0.18631303673925995
346300000 0.14019494780638814
346400000 0.18351278649758962
346500000 0.15068374310347768
346600000 0.1621533519949615
346700000 0.15569386479849284
346800000 0.12492339288302924
346900000 0.15198527044389645
347000000 0.12344884450055825
347100000 0.15389513267750543
347200000 0.13760532850557566
347300000 0.14183489868407118
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.08631514408338072
347500000 0.01904355505129695
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of this type: 1107
best so far: 0
type: [1, 41, 3, 1, 1] 123
cas